# Building a Basic CNN: The MNIST Dataset

In this notebook, we will build a simple CNN-based architecture to classify the 10 digits (0-9) of the MNIST dataset. The objective of this notebook is to become familiar with the process of building CNNs in Keras.

We will go through the following steps:
1. Importing libraries and the dataset
2. Data preparation: Train-test split, specifying the shape of the input data etc.
3. Building and understanding the CNN architecture 
4. Fitting and evaluating the model

Let's dive in.

### 1. Importing libraries and dataset

In [11]:
import numpy as np
import keras 
import random
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import MaxPooling2D, Conv2D
from keras import backend as K

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 21s 2us/step


In [9]:
print("Training data shape: x: ", x_train.shape, " Y: ", y_train.shape)
print("Testing data shape: x: ", x_test.shape, " Y: ", y_test.shape)

Training data shape: x:  (60000, 28, 28)  Y:  (60000,)
Testing data shape: x:  (10000, 28, 28)  Y:  (10000,)


In [13]:
idx = np.random.randint(x_train.shape[0], size=20000)
x_train = x_train[idx, :]
y_train = y_train[idx]

In [14]:
print("Training data shape: x: ", x_train.shape, " Y: ", y_train.shape)

Training data shape: x:  (20000, 28, 28)  Y:  (20000,)


## 2. Data Preparation

Let's prepare the dataset for feeding to the network. We will do the following three main steps:<br>

#### 2.1 Reshape the Data
First, let's understand the shape in which the network expects the training data. 
Since we have 20,000 training samples each of size (28, 28, 1), the training data (`x_train`) needs to be of the shape `(20000, 28, 28, 1)`. If the images were coloured, the shape would have been `(20000, 28, 28, 3)`.

Further, each of the 20,000 images have a 0-9 label, so `y_train` needs to be of the shape `(20000, 10)` where each image's label is represented as a 10-d **one-hot encoded vector**.

The shapes of `x_test` and `y_test` will be the same as that of `x_train` and `y_train` respectively.

#### 2.2 Rescaling (Normalisation)
The value of each pixel is between 0-255, so we will **rescale each pixel** by dividing by 255 so that the range becomes 0-1. Recollect <a href="https://stats.stackexchange.com/questions/185853/why-do-we-need-to-normalize-the-images-before-we-put-them-into-cnn">why normalisation is important for training NNs</a>.

#### 2.3 Converting Input Data Type: Int to Float
The pixels are originally stored as type `int`, but it is advisable to feed the data as `float`. This is not really compulsory, but advisable. You can read <a href="https://datascience.stackexchange.com/questions/13636/neural-network-data-type-conversion-float-from-int">why conversion from int to float is helpful here</a>.


In [15]:
img_row_size = 28
img_col_size = 28

batch_size = 128
num_classes = 10
epochs = 12

In [16]:
x_train = x_train.reshape(x_train.shape[0], img_row_size, img_col_size, 1)
y_train = keras.utils.to_categorical(y_train, num_classes)

x_test = x_test.reshape(x_test.shape[0], img_row_size, img_col_size, 1)
y_test = keras.utils.to_categorical(y_test, num_classes)

print("Training data shape: x: ", x_train.shape, " Y: ", y_train.shape)
print("Testing data shape: x: ", x_test.shape, " Y: ", y_test.shape)

Training data shape: x:  (20000, 28, 28, 1)  Y:  (20000, 10)
Testing data shape: x:  (10000, 28, 28, 1)  Y:  (10000, 10)


In [17]:
# ===> SCALING: 

x_train = x_train/255
x_test = x_test/255

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

### 3. Building the Model: 

Let's now build the CNN architecture. For the MNIST dataset, we do not need to build a very sophisticated CNN - a simple shallow-ish CNN would suffice. 

We will build a network with:
- two convolutional layers having 32 and 64 filters respectively, 
- followed by a max pooling layer, 
- and then `Flatten` the output of the pooling layer to give us a long vector, 
- then add a fully connected `Dense` layer with 128 neurons, and finally
- add a `softmax` layer with 10 neurons

The generic way to build a model in Keras is to instantiate a `Sequential` model and keep adding `keras.layers` to it. We will also use some dropouts.

In [19]:
help(Sequential())

Help on Sequential in module keras.engine.sequential object:

class Sequential(keras.engine.training.Model)
 |  Sequential(layers=None, name=None)
 |  
 |  Linear stack of layers.
 |  
 |  # Arguments
 |      layers: list of layers to add to the model.
 |  
 |  # Example
 |  
 |  ```python
 |  # Optionally, the first layer can receive an `input_shape` argument:
 |  model = Sequential()
 |  model.add(Dense(32, input_shape=(500,)))
 |  
 |  # Afterwards, we do automatic shape inference:
 |  model.add(Dense(32))
 |  
 |  # This is identical to the following:
 |  model = Sequential()
 |  model.add(Dense(32, input_dim=500))
 |  
 |  # And to the following:
 |  model = Sequential()
 |  model.add(Dense(32, batch_input_shape=(None, 500)))
 |  
 |  # Note that you can also omit the `input_shape` argument:
 |  # In that case the model gets built the first time you call `fit` (or other
 |  # training and evaluation methods).
 |  model = Sequential()
 |  model.add(Dense(32))
 |  model.add(Dense(32

In [21]:
input_shape = (img_row_size, img_col_size, 1)
model = Sequential()

model.add(Conv2D(32, input_shape=input_shape, kernel_size=(3, 3), activation='relu'))

model.add(Conv2D(62, input_shape=input_shape, kernel_size=(3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

# ===> After this put a fully connected layer: 

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

# model summary
model.summary()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 62)        17918     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 62)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 62)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8928)              0         
_________________________________________________________________
dense_1 (Dense)      

#### Understanding Model Summary

It is a good practice to spend some time staring at the model summary above and verify the number of parameteres, output sizes etc. Let's do some calculations to verify that we understand the model deeply enough. 

- Layer-1 (Conv2D): We have used 32 kernels of size (3, 3), and each kernel has a single bias, so we have 32 x 3 x 3 (weights) + 32 (biases) = 320 parameters (all trainable). Note that the kernels have only one channel since the input images are 2D (grayscale). By default, a convolutional layer uses stride of 1 and no padding, so the output from this layer is of shape 26 x 26 x 32, as shown in the summary above (the first element `None` is for the batch size).

- Layer-2 (Conv2D): We have used 64 kernels of size (3, 3), but this time, each kernel has to convolve a tensor of size (26, 26, 32) from the previous layer. Thus, the kernels will also have 32 channels, and so the shape of each kernel is (3, 3, 32) (and we have 64 of them). So we have 64 x 3 x 3 x 32 (weights) + 64 (biases) = 18496 parameters (all trainable). The output shape is (24, 24, 64) since each kernel produces a (24, 24) feature map.

- Max pooling: The pooling layer gets the (24, 24, 64) input from the previous conv layer and produces a (12, 12, 64) output (the default pooling uses stride of 2). There are no trainable parameters in the pooling layer.

- The `Dropout` layer does not alter the output shape and has no trainable parameters.

- The `Flatten` layer simply takes in the (12, 12, 64) output from the previous layer and 'flattens' it into a vector of length 12 x 12 x 64 = 9216.

- The `Dense` layer is a plain fully connected layer with 128 neurons. It takes the 9216-dimensional output vector from the previous layer (layer l-1) as the input and has 128 x 9216 (weights) + 128 (biases) =  1179776 trainable parameters. The output of this layer is a 128-dimensional vector.

- The `Dropout` layer simply drops a few neurons.

- Finally, we have a `Dense` softmax layer with 10 neurons which takes the 128-dimensional vector from the previous layer as input. It has 128 x 10 (weights) + 10 (biases) = 1290 trainable parameters.

Thus, the total number of parameters are 1,199,882 all of which are trainable.

## 4. Fitting and Evaluating the Model

Let's now compile and train the model.

In [24]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

In [25]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), verbose=True)

Instructions for updating:
Use tf.cast instead.
Train on 20000 samples, validate on 10000 samples
Epoch 1/12
20000/20000 [==============================] - 43s 2ms/step - loss: 0.4737 - acc: 0.8527 - val_loss: 0.1193 - val_acc: 0.9644
Epoch 2/12
20000/20000 [==============================] - 43s 2ms/step - loss: 0.1478 - acc: 0.9570 - val_loss: 0.0841 - val_acc: 0.9736
Epoch 3/12
20000/20000 [==============================] - 40s 2ms/step - loss: 0.1042 - acc: 0.9697 - val_loss: 0.0644 - val_acc: 0.9801
Epoch 4/12
20000/20000 [==============================] - 61s 3ms/step - loss: 0.0778 - acc: 0.9761 - val_loss: 0.0573 - val_acc: 0.9813
Epoch 5/12
20000/20000 [==============================] - 47s 2ms/step - loss: 0.0640 - acc: 0.9813 - val_loss: 0.0546 - val_acc: 0.9826
Epoch 6/12
20000/20000 [==============================] - 2157s 108ms/step - loss: 0.0529 - acc: 0.9838 - val_loss: 0.0501 - val_acc: 0.9833
Epoch 7/12
20000/20000 [==============================] - 47s 2ms/step - los

In [26]:
print(model.evaluate(x_test, y_test))
print(model.metrics_names)

10000/10000 [==============================] - 6s 554us/step
[0.047754203886926916, 0.9862]
['loss', 'acc']
